<a href="https://colab.research.google.com/github/SwethaKondapalli02907/Fmml-/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [15]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [16]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-16-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [17]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [18]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [19]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [20]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [21]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [22]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [23]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [24]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [25]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [26]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

answers

1. The percentage of the validation set has a direct impact on the accuracy of the model. If we increase the percentage of the validation set, we reduce the number of samples in the training set. This can lead to overfitting and poor generalization of the model. In other words, the model may perform well on the training data but poorly on new data 1.

On the other hand, if we reduce the percentage of the validation set, we increase the number of samples in the training set. This can lead to underfitting and poor performance on both training and validation data 2.

A common practice is to use a validation set that is 20% of the total dataset 2. However, there is no one-size-fits-all solution, and it depends on various factors such as dataset size, complexity, and model architecture 12.



2.  The size of the training and validation sets can have an impact on the accuracy of the model on the test set. If the validation set is too small, then the accuracy, precision, recall, and F1 score could have a large variance. You might happen to get a really lucky or a really unlucky split! In general, putting 80% of your data in the training set, and 20% of your data in the validation set is a good place to start 1.

On the other hand, if you have a small training set, then your model may not learn the underlying distribution of the real data well. In this case, increasing the size of the training set can help improve performance on both training and test sets 2.



3. .There is no one-size-fits-all solution for the percentage of the validation set. However, a common practice is to use a validation set that is 20% of the total dataset 1. This split is a good starting point, but it depends on various factors such as dataset size, complexity, and model architecture 21.

If you have a small validation set, then the accuracy, precision, recall, and F1 score could have a large variance. In general, putting 80% of your data in the training set, and 20% of your data in the validation set is a good place to start 1.

On the other hand, if you have a small training set, then your model may not learn the underlying distribution of the real data well. In this case, increasing the size of the training set can help improve performance on both training and test sets 3.

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [27]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [28]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

answers

1.  Yes, averaging the validation accuracy across multiple splits can give more consistent results. This is because the validation set is randomly split from the training set, and different splits can lead to different results. By averaging the validation accuracy across multiple splits, we can get a more accurate estimate of the model’s performance on unseen data .

In general, k-fold cross-validation is a common technique used to estimate the performance of a model. In k-fold cross-validation, the data is split into k-folds, and the model is trained and evaluated k times. Each time, a different fold is used as the validation set, and the remaining folds are used as the training set. The validation accuracy is then averaged across all k-folds .



2.  Yes, averaging the validation accuracy across multiple splits can give a more accurate estimate of the model’s performance on unseen data. This is because the validation set is randomly split from the training set, and different splits can lead to different results. By averaging the validation accuracy across multiple splits, we can get a more accurate estimate of the model’s performance on unseen data .

However, it’s important to note that cross-validation is not a substitute for testing on a completely independent dataset. Cross-validation only provides an estimate of how well the model will perform on new data from the same distribution as the training data


3.  The effect of the number of iterations on an estimate depends on the specific estimation method being used. In many cases, increasing the number of iterations can lead to a more accurate estimate, but there may be diminishing returns.

For example, in iterative numerical methods like the Monte Carlo simulation or gradient descent, increasing the number of iterations can often lead to a better estimate of a value or a function minimum, respectively. This is because more iterations allow the method to refine the estimate and approach the true value.

However, there is a trade-off to consider. Increasing the number of iterations also increases computational time and resource usage. At some point, the improvement in the estimate becomes marginal, and it may not be worth the added computational cost.

In summary, higher iterations can generally lead to better estimates, but it's essential to balance accuracy with computational resources when deciding on the number of iterations to use in a specific estimation method.


4.   Increasing the number of iterations can help improve the performance of machine learning models when dealing with small training or validation datasets to some extent. However, it's important to understand the limitations and potential issues that may arise:

Overfitting: If you have a very small dataset, increasing the number of iterations excessively can lead to overfitting. The model might start memorizing the limited training data instead of learning meaningful patterns, resulting in poor generalization to new, unseen data.

Regularization: Use regularization techniques (e.g., L1, L2 regularization) to prevent overfitting when working with small datasets.

Feature Engineering: Carefully design and select relevant features to make the most of the available data.

Transfer Learning: If applicable, leverage pre-trained models and fine-tune them on your small dataset.

In conclusion, while increasing iterations can help to some extent with small datasets, it's not a panacea. It's crucial to consider other strategies and strike a balance between model complexity and dataset size to achieve the best results.
